In [28]:
#pip install -U jupyter ipywidgets
#pip install "huggingface_hub[hf_xet]"


In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, platform, os, sys

print("CUDA:", torch.cuda.is_available(), torch.cuda.get_device_name(0))
print("Python:", platform.python_version())


CUDA: True NVIDIA GeForce RTX 3060
Python: 3.10.16


In [50]:
# ==============================================================
# 0. ライブラリ
# ==============================================================

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    StoppingCriteria,
    StoppingCriteriaList,
)
import transformers, torch, textwrap, platform

# ==============================================================
# 1. モデル読み込み（RTX-3060／12 GB 用の最軽量構成）
# ==============================================================

model_id = "Qwen/Qwen1.5-7B-Chat"          # 7B-Chat を 8-bit でロード
bnb_cfg = BitsAndBytesConfig(load_in_8bit=True)

tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    max_memory={0: "10GiB"},               # CUDA OOM ガード
    quantization_config=bnb_cfg,
    trust_remote_code=True,
)

print("GPU :", torch.cuda.get_device_name(0))
print("torch", torch.__version__, "transformers", transformers.__version__)

# ==============================================================
# 2. Stop 句（「\n\n===」または <|im_end|> で終了）
# ==============================================================

class StopOnSeq(StoppingCriteria):
    def __init__(self, stop_ids_list):
        super().__init__()
        self.stop_ids_list = stop_ids_list

    def __call__(self, input_ids: torch.LongTensor, scores, **kwargs):
        for stop_ids in self.stop_ids_list:
            if (
                input_ids.shape[1] >= len(stop_ids)
                and torch.equal(input_ids[0, -len(stop_ids):], stop_ids)
            ):
                return True
        return False


stop_ids  = tok("<|im_end|>", add_special_tokens=False)["input_ids"]
stop_ids2 = tok("\n\n===", add_special_tokens=False)["input_ids"]
stopping  = StoppingCriteriaList([StopOnSeq([torch.tensor(stop_ids,  device=model.device),
                                             torch.tensor(stop_ids2, device=model.device)])])

# ==============================================================
# 3. few-shot 例（new_request / reply 各 1 本だけ）
# ==============================================================

few_new_user = "プロジェクトAのレポート作成を依頼してください。"
few_new_assi = textwrap.dedent("""\
    ### Step1 AIエージェントからユーザーへの受け止め（ai_receipt）
    依頼内容を整理する姿勢が素晴らしいですね。
    ### Step2 改善案の考え方（thinking）
    目的・期日・支援体制をセットで示すと相手が動きやすくなります。
    ### Step3 改善案（suggestion）
    お疲れさまです。プロジェクトAの経過レポートを **来週月曜 12:00** までにご作成いただけますか？\
    不明点があれば遠慮なくお知らせください。よろしくお願いします。
""")

few_rep_user = "部下が『お客様に納期遅延の連絡をしてもいいですか？』と投稿。あなたは『なぜ遅れた？』と返信しようとしている。"
few_rep_assi = textwrap.dedent("""\
    ### Step1 AIエージェントからユーザーへの受け止め（ai_receipt）
    納期遅延はご心配ですよね。迅速に状況を把握したいお気持ち、理解できます。
    ### Step2 改善案の考え方（thinking）
    まず報告を評価し、事実確認→対策提案の順にすると報告の質が上がります。
    ### Step3 改善案（suggestion）
    ご報告ありがとう。状況を整理して一緒に対応したいので、現状と原因を共有してもらえますか？\
    そのうえでお客様への説明方法を考えましょう。
""")

few_block = (
    f"<|im_start|>user\n{few_new_user}\n<|im_end|>\n"
    f"<|im_start|>assistant\n{few_new_assi}\n<|im_end|>\n"
    f"<|im_start|>user\n{few_rep_user}\n<|im_end|>\n"
    f"<|im_start|>assistant\n{few_rep_assi}\n<|im_end|>\n"
)

# ==============================================================
# 4. 生成関数
# ==============================================================

def senpai_feedback(mode: str, manager_draft: str, thread_context: str = "") -> str:
    """
    mode           : "new_request" | "reply"
    manager_draft  : ユーザが送ろうとしている下書き
    thread_context : reply モード時の直近スレッド全文
    """
    assert mode in {"new_request", "reply"}

    system_msg = (
        "あなたは親切で思いやりのあるビジネスメンターです。"
        "Slack/Teams の日本語チャットを改善します。"
        "中国語・英語は禁止。出力は必ず **丁寧な日本語** の Markdown で、"
        "以下 3 見出しのみを用いてください：\n"
        "### Step1 AIエージェントからユーザーへの受け止め（ai_receipt）\n"
        "### Step2 改善案の考え方（thinking）\n"
        "### Step3 改善案（suggestion）\n"
    )

    if mode == "new_request":
        user_prompt = f"[コンテキスト:new_request]\n{manager_draft}"
    else:
        user_prompt = (
            "[コンテキスト:reply]\n"
            f"[直近スレッド]\n{thread_context}\n"
            f"[あなたの返信案]\n{manager_draft}"
        )

    prompt = (
        f"<|im_start|>system\n{system_msg}\n<|im_end|>\n"
        f"{few_block}"
        f"<|im_start|>user\n{user_prompt}\n<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    out = model.generate(
        **tok(prompt, return_tensors="pt").to(model.device),
        max_new_tokens=256,
        temperature=0.1,
        top_p=0.35,
        repetition_penalty=1.5,
        stopping_criteria=stopping,
    )

    text = tok.decode(out[0], skip_special_tokens=True)
    answer = text.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
    return answer



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GPU : NVIDIA GeForce RTX 3060
torch 2.1.2+cu118 transformers 4.52.3


In [51]:
# ==============================================================
# 5. 動作テスト
# ==============================================================

if __name__ == "__main__":
    nr_example = "Cさん、来週のプレゼン資料をまとめておいて！"
    rp_thread  = "部下: クライアントから急ぎ質問が来ました。どう返答すべきでしょう？"
    rp_draft   = "そっちで考えて先に返しておいてよ。"

    print("=== new_request ===")
    print(senpai_feedback("new_request", nr_example))

    print("\n=== reply ===")
    print(senpai_feedback("reply", rp_draft, thread_context=rp_thread))


=== new_request ===
system
あなたは親切で思いやりのあるビジネスメンターです。Slack/Teams の日本語チャットを改善します。中国語・英語は禁止。出力は必ず **丁寧な日本語** の Markdown で、以下 3 見出しのみを用いてください：
### Step1 AIエージェントからユーザーへの受け止め（ai_receipt）
### Step2 改善案の考え方（thinking）
### Step3 改善案（suggestion）


user
プロジェクトAのレポート作成を依頼してください。

assistant
### Step1 AIエージェントからユーザーへの受け止め（ai_receipt）
依頼内容を整理する姿勢が素晴らしいですね。
### Step2 改善案の考え方（thinking）
目的・期日・支援体制をセットで示すと相手が動きやすくなります。
### Step3 改善案（suggestion）
お疲れさまです。プロジェクトAの経過レポートを **来週月曜 12:00** までにご作成いただけますか？    不明点があれば遠慮なくお知らせください。よろしくお願いします。


user
部下が『お客様に納期遅延の連絡をしてもいいですか？』と投稿。あなたは『なぜ遅れた？』と返信しようとしている。

assistant
### Step1 AIエージェントからユーザーへの受け止め（ai_receipt）
納期遅延はご心配ですよね。迅速に状況を把握したいお気持ち、理解できます。
### Step2 改善案の考え方（thinking）
まず報告を評価し、事実確認→対策提案の順にすると報告の質が上がります。
### Step3 改善案（suggestion）
ご報告ありがとう。状況を整理して一緒に対応したいので、現状と原因を共有してもらえますか？    そのうえでお客様への説明方法を考えましょう。


user
[コンテキスト:new_request]
Cさん、来週のプレゼン資料をまとめておいて！

assistant
### Step1 AIエージェентからのメッセージ (new_request)
【新規リクスタ】 C桑, 来週のお話しのためにpresentation材料をお作りいただきたいと思います！

### Step